In [1]:
import tensorflow as tf

# import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# tensorflow.contrib.learn.python.learn.datasets.base.Datasets
# with batch function
mnist = input_data.read_data_sets("../basics/MNIST/MNIST_data", one_hot=True)

Extracting ../basics/MNIST/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../basics/MNIST/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../basics/MNIST/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../basics/MNIST/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
lr = 0.01
b_size = 100
dis_step = 1
model_path = 'tmp/model.ckpt'

# net parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# create model
def multilayer_perception(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# store
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perception(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

init = tf.global_variables_initializer()

In [3]:
saver = tf.train.Saver()

In [6]:
print "starting 1st session .."
with tf.Session() as sess:
    sess.run(init)
    for ep in range(3):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/b_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(b_size)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        if ep % dis_step == 0:
            print "Epoch:", '%04d' % (ep+1), "cost=", "{:.9f}".format(avg_cost)
    print '1st optimization finished.'
    
    # test
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
    
    # save
    save_path = saver.save(sess, model_path)
    print "model saved in file: %s" % save_path

starting 1st session ..
Epoch: 0001 cost= 51.486238579
Epoch: 0002 cost= 8.757420696
Epoch: 0003 cost= 4.567288167
1st optimization finished.
Accuracy: 0.9478
model saved in file: tmp/model.ckpt


In [10]:
print 'starting 2nd session ..'
with tf.Session() as sess:
    sess.run(init)    # always initialization
    
    load_path = saver.restore(sess, model_path)
    print "model restored from file: %s" % save_path
    
    for ep in range(7):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/b_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(b_size)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c/total_batch
        if ep % dis_step == 0:
            print "Epoch:", '%04d' % (ep + 1), "cost=", "{:.9f}".format(avg_cost)
    print '2nd optimization finished.'
        
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

starting 2nd session ..
model restored from file: tmp/model.ckpt
Epoch: 0001 cost= 3.112969797
Epoch: 0002 cost= 2.599156688
Epoch: 0003 cost= 2.344627583
Epoch: 0004 cost= 1.972812705
Epoch: 0005 cost= 1.823774524
Epoch: 0006 cost= 1.495449158
Epoch: 0007 cost= 1.189701302
2nd optimization finished.
Accuracy: 0.9572
